# Cam Recognition Driver

In [ ]:
# # file op example for notes
# def logVisitor(message):
#     with open("sampleImages/LockCamLogs.txt","a") as l:
#         l.write(message+"\n \n")
#         l.close()
# logVisitor("test message1") 

In [ ]:
# def extractFiles(folderPath, folderName, subFolder=''):
#     # Setup path variables
#     copyName = folderName+'_Extractable_'+(time.strftime("%m%d%y_%H%M%S", time.gmtime()))
#     copyPath = folderPath+copyName
#     zipPath = folderPath+subFolder+copyName
#     transferredFiles = glob.glob(folderPath+folderName+'/*')

#     # Copy files from src to destination
#     shutil.copytree(folderName, copyPath)

#     # Zip temporary files into target location
#     try:
#         shutil.make_archive(zipPath, 'zip', folderPath, copyName)
#     except OSError:
#         pass

#     # Clean up transferred Images
#     for f in transferredFiles:
#         os.remove(f)

#     # Clean up tempory file
#     shutil.rmtree(copyPath)
#     return

# extractFiles('/home/pi/LockCam/','sampleImages','extraction/')

In [55]:
import numpy as np
import cv2
from IPython.display import clear_output

import tensorflow as tf
# import tflite as tf
# import tflite_runtime.interpreter as tflite


lockStatus = 1
conThres = 0.95
lockTimeOut = 10

# Load TFLite model and allocate tensors.
# on tensorflow lib

#interpreter = tf.lite.Interpreter(model_path='savedTFLite3')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb1TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb2TFlite')
interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb3TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb4TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb5TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb7TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb8TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionb9TFlite')
#interpreter = tf.lite.Interpreter(model_path='lockIndust001FactialRecognitionBETA1TFlite')


# # on tflite lib
#interpreter = tflite.Interpreter(model_path="savedTFLite3.tflite")

## temp example
# # docs_infra: no_execute
#interpreter = tf.lite.Interpreter(model_content='savedTFLite3.tflite')

interpreter.allocate_tensors()
    
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(output_details)
if output_details[0]['shape'][1] == 8:
    print('8 Classes')
    class_labels = ['Dale','Jameda','Joey','Kylie','Michael','Patricia','Tony','Violet']
    print('Dale','Jameda','Joey','Kylie','Michael','Patricia','Tony','Violet')
if output_details[0]['shape'][1] == 9:
    print('8 Classes')
    class_labels = ['Null','Dale','Jameda','Joey','Kylie','Michael','Patricia','Tony','Violet']
    print('Null','Dale','Jameda','Joey','Kylie','Michael','Patricia','Tony','Violet')


[{'name': 'Identity', 'index': 188, 'shape': array([1, 8]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
8 Classes
Dale Jameda Joey Kylie Michael Patricia Tony Violet


### Defines lots of functions

In [56]:
def lockDoor():
    global lockStatus
    print("Locking Door!")
    placeholder2 = 2
    lockStatus = 1
        

In [57]:
def unlockDoor():
    global lockStatus
    print("Unlocking Door!")
    placeholder1 = 1
    #print(lockStatus)
    lockStatus = 0
    #print(lockStatus)
    

In [58]:
def checkLock():
    global lockStatus
    global lockTimeOut
    #print("checking lock")
    print("lockStatus = ", lockStatus)
    print("lockTimeOut", lockTimeOut)
    if lockStatus == 0:
        lockTimeOut -= 1
        if lockTimeOut < 0:
            lockDoor()
            

In [59]:
def operateLock():
    global lockStatus
    global lockTimeOut
    lockTimeOut = 10 # Resets everytime operateLock is called
    #print(lockStatus)
    if lockStatus == 1:
        unlockDoor()
    else:
        print("Door is Unlocked")
        

In [60]:
# This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
def lite_model(img):    
    input_shape = input_details[0]['shape']

    input_data = np.array(img, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    #interpreter.set_tensor(interpreter.get_input_details()[0]['index'], images)

    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index'])
    #return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])
    

In [61]:
def requestResults(resultSet):
    predicted_index = np.argmax(result_array, axis=-1)
    prediction_strength = result_array[0][predicted_index]
    if prediction_strength >= conThres:
        predicted_label = class_labels[int(predicted_index)]
        clear_output(wait=True) #True)
        print('result_array = ', resultSet)
        print('predicted_index = ', predicted_index)
        print('prediction_strength = ', prediction_strength)
        print('predicted_label = ', predicted_label)
        #print('currentframe = ', currentframe)
        if predicted_label != 'Null':
            operateLock()
        
    else:
        #if currentframe % (3*capRatio) == 0:
            clear_output(wait=True)
            print('result_array = ')
            print('predicted_index = ')
            print('prediction_strength = ')
            print('predicted_label = ')
            #print('currentframe = ', currentframe)

#### Here we get the video feed and send it for inference in a loop

In [62]:
capRatio = 25
currentframe = 0

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    if currentframe %capRatio == 0:
        #print('frame.shape = ', frame.shape)
        pipeFrame = cv2.resize(frame, (224, 224))
        pipeFrame=pipeFrame / 255.0
        #print('pipeFrame.shape = ', pipeFrame.shape)
        #cv2.imshow('pipeFrame', pipeFrame)
        expPipeFrame = np.expand_dims(pipeFrame, axis=0)
        #print('expPipeFrame.shape = ', expPipeFrame.shape)
        result_array = lite_model(expPipeFrame)
        requestResults(result_array)
        
        checkLock()
    
    # Display the resulting frame
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    currentframe += 1
    #print(currentframe)
    if currentframe >= 1000000000000:
        currentframe = 1

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

result_array =  [[0.00390625 0.00390625 0.00390625 0.         0.9765625  0.00390625
  0.         0.00390625]]
predicted_index =  [4]
prediction_strength =  [0.9765625]
predicted_label =  Michael
Door is Unlocked
lockStatus =  0
lockTimeOut 10


# You should see output updating right here ^ with announcements of any activity!